In [39]:
import random
import json
import numpy as np
import pickle
import tensorflow as tf
import pandas as pd
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [40]:
intents = json.loads(open('./intents.json').read())
intents

{'intents': [{'tag': 'greetings',
   'patterns': ['hello',
    'hai',
    'hi',
    'hey',
    'halo',
    'good day',
    'helo',
    'hallo',
    'good morning',
    'good afternoon',
    'good night'],
   'responses': ['Hello',
    'Hi',
    'What can i do for you?',
    'Welcome',
    'can i help you?']},
  {'tag': 'kabar',
   'patterns': ['How your day?',
    'How do you do?',
    'Whassup?',
    'How are you doing?',
    'halo',
    'good day',
    'How are you?'],
   'responses': ['Not so bad. How about you?',
    "I'm great, thanks!, you?",
    "We're great",
    "I'm fine, thanks. How about you? ",
    'Very well. How about you? How are you today?']},
  {'tag': 'kabar_baik',
   'patterns': ['Not so bad',
    'Im great',
    'Were great',
    'Im fine',
    'Very well',
    'Not bad',
    'Pretty good',
    'Fantastic',
    'Wonderful',
    'Im good',
    'good'],
   'responses': ['Oh, great. What can i do for you?',
    'Thanks God, May I ...?',
    'Thanks God, Would you like

In [41]:
'intents.json-> intents-> {tag, patterns, responses}' "ket: -> berarti berisi"

'intents.json-> intents-> {tag, patterns, responses}ket: -> berarti berisi'

In [42]:
intents_df = pd.read_json('./intents.json') 

In [43]:
intents_df.head() #hasil pd.read_json tidak sesuai format dataframe yg diinginkan

,intents
0,"{'tag': 'greetings', 'patterns': ['hello', 'ha..."
1,"{'tag': 'kabar', 'patterns': ['How your day?',..."
2,"{'tag': 'kabar_baik', 'patterns': ['Not so bad..."
3,"{'tag': 'kabar_buruk', 'patterns': ['I have Be..."
4,"{'tag': 'goodbye', 'patterns': ['bye', 'cya', ..."


In [44]:
# Flatten data, json to pandas dataframe
df_chat = pd.json_normalize(intents, record_path =['intents'])
df_chat = df_chat[['tag', 'patterns']]
df_chat = df_chat.explode(['patterns'], ignore_index=True)
df_chat = pd.DataFrame(df_chat)
df_chat

,tag,patterns
0,greetings,hello
1,greetings,hai
2,greetings,hi
3,greetings,hey
4,greetings,halo
...,...,...
56,thank,thank you
57,thank,thank u
58,thank,thanks
59,thank,trims


In [45]:
def remove_unused(text):
    text = re.sub('[0-9]+', '', text) #untuk menghilangkan angka
    #untuk menghilangkan non-ASCII characters dan unicode
    text = text.encode('ascii', 'ignore').decode('utf-8') 
    text = re.sub(r'[^\x00-\x7f]', r'', text)  
    text = re.sub(r'[^\w]', ' ', text) #untuk menghiilangkan selain alpha numerik
    #untuk menghilangkan double atau lebih spasi
    space = ['    ', '   ', '  ']
    for i in space:
        text = text.replace(i, ' ')
    text = text.lower().strip()
#     text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [46]:
df_chat['cleantext'] = df_chat['patterns'].apply(lambda x: remove_unused(x))

In [47]:
tokenizer = word_tokenize
lemmatizer = WordNetLemmatizer()

In [48]:
def lemm(word):
    words = tokenizer(word)
    words = [lemmatizer.lemmatize(text) for text in words]
    words = [" ".join(t for t in words)]
    return words

In [49]:
df_chat['cleantext'].apply(lemm)

0                 [hello]
1                   [hai]
2                    [hi]
3                   [hey]
4                  [halo]
             ...         
56            [thank you]
57              [thank u]
58               [thanks]
59                 [trim]
60    [thank you so much]
Name: cleantext, Length: 61, dtype: object

# Target Enginering

In [50]:
classes = sorted(df_chat['tag'].unique())
classes

['age',
 'goodbye',
 'greetings',
 'kabar',
 'kabar_baik',
 'kabar_buruk',
 'name',
 'thank']

In [51]:
df_chat[classes] = pd.get_dummies(df_chat['tag'], drop_first=False)
df_chat

,tag,patterns,cleantext,age,goodbye,greetings,kabar,kabar_baik,kabar_buruk,name,thank
0,greetings,hello,hello,0,0,1,0,0,0,0,0
1,greetings,hai,hai,0,0,1,0,0,0,0,0
2,greetings,hi,hi,0,0,1,0,0,0,0,0
3,greetings,hey,hey,0,0,1,0,0,0,0,0
4,greetings,halo,halo,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
56,thank,thank you,thank you,0,0,0,0,0,0,0,1
57,thank,thank u,thank u,0,0,0,0,0,0,0,1
58,thank,thanks,thanks,0,0,0,0,0,0,0,1
59,thank,trims,trims,0,0,0,0,0,0,0,1


In [52]:
dataset_df = df_chat.drop(['tag', 'patterns'], axis=1)

In [53]:
dataset_df.head()

,cleantext,age,goodbye,greetings,kabar,kabar_baik,kabar_buruk,name,thank
0,hello,0,0,1,0,0,0,0,0
1,hai,0,0,1,0,0,0,0,0
2,hi,0,0,1,0,0,0,0,0
3,hey,0,0,1,0,0,0,0,0
4,halo,0,0,1,0,0,0,0,0


In [54]:
dataset_df.shape

(61, 9)

In [55]:
X_df = dataset_df['cleantext']
y_df = dataset_df.drop('cleantext', axis= 1)

In [56]:
print(X_df.shape)
print(y_df.shape) #Target = y_df = 8

(61,)
(61, 8)


# Feature Enginering Bag Of Word

In [57]:
from sklearn.feature_extraction.text import CountVectorizer

In [58]:
vectorizer_bow= CountVectorizer()

In [59]:
print(y_df.keys())

Index(['age', 'goodbye', 'greetings', 'kabar', 'kabar_baik', 'kabar_buruk',
       'name', 'thank'],
      dtype='object')


In [60]:
X_df.values

array(['hello', 'hai', 'hi', 'hey', 'halo', 'good day', 'helo', 'hallo',
       'good morning', 'good afternoon', 'good night', 'how your day',
       'how do you do', 'whassup', 'how are you doing', 'halo',
       'good day', 'how are you', 'not so bad', 'im great', 'were great',
       'im fine', 'very well', 'not bad', 'pretty good', 'fantastic',
       'wonderful', 'im good', 'good', 'i have been better', 'bad day',
       'things are not so good', 'i am feeling sad today',
       'i just feel tired', 'bad', 'not good', 'sad', 'bye', 'cya',
       'see you later', 'daa', 'good to see you', 'please to meet you',
       'nice to meet you', 'how old are you', 'how old r u', 'how old',
       'your age', 'age', 'tell me your name', 'what is your name',
       'who are you', 'your name', 'do you have a name',
       'what should i call you', 'tell me your name', 'thank you',
       'thank u', 'thanks', 'trims', 'thank you so much'], dtype=object)

In [61]:
X_train_vec = vectorizer_bow.fit_transform(X_df.values)
X_train_vec

<61x65 sparse matrix of type '<class 'numpy.int64'>'
	with 136 stored elements in Compressed Sparse Row format>

In [62]:
pickle.dump(vectorizer_bow, open('260823_CountVecBoW_CBot.pkl', 'wb'))

In [63]:
len(vectorizer_bow.vocabulary_)

65

In [64]:
vectorizer_bow.vocabulary_

{'hello': 24,
 'hai': 20,
 'hi': 27,
 'hey': 26,
 'halo': 22,
 'good': 18,
 'day': 11,
 'helo': 25,
 'hallo': 21,
 'morning': 35,
 'afternoon': 0,
 'night': 39,
 'how': 28,
 'your': 64,
 'do': 12,
 'you': 63,
 'whassup': 59,
 'are': 3,
 'doing': 13,
 'not': 40,
 'so': 47,
 'bad': 4,
 'im': 29,
 'great': 19,
 'were': 58,
 'fine': 17,
 'very': 56,
 'well': 57,
 'pretty': 43,
 'fantastic': 14,
 'wonderful': 62,
 'have': 23,
 'been': 5,
 'better': 6,
 'things': 51,
 'am': 2,
 'feeling': 16,
 'sad': 44,
 'today': 54,
 'just': 31,
 'feel': 15,
 'tired': 52,
 'bye': 7,
 'cya': 9,
 'see': 45,
 'later': 32,
 'daa': 10,
 'to': 53,
 'please': 42,
 'meet': 34,
 'nice': 38,
 'old': 41,
 'age': 1,
 'tell': 48,
 'me': 33,
 'name': 37,
 'what': 60,
 'is': 30,
 'who': 61,
 'should': 46,
 'call': 8,
 'thank': 49,
 'thanks': 50,
 'trims': 55,
 'much': 36}

In [65]:
y_train = np.array(y_df)#.to_numpy()

In [66]:
type(y_train)

numpy.ndarray

In [67]:
type(X_train_vec)

scipy.sparse._csr.csr_matrix

In [68]:
print(X_train_vec.shape)
print(y_df.shape)

(61, 65)
(61, 8)


In [69]:
X_train = X_train_vec.toarray()

In [70]:
print(X_train.shape)
print(y_train.shape)

(61, 65)
(61, 8)


In [71]:
print(len(X_train[0]))
print(len(y_train[0]))

65
8


In [72]:
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation= 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

In [73]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               8448      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 8)                 520       
                                                                 
Total params: 17224 (67.28 KB)
Trainable params: 17224 (67.28 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [74]:
sgd= SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [75]:
model.fit(X_train, y_train, epochs=500, verbose=1)

Epoch 1/500
2/2 [==============================] - 1s 8ms/step - loss: 2.0932 - accuracy: 0.1475
Epoch 2/500
2/2 [==============================] - 0s 7ms/step - loss: 2.1026 - accuracy: 0.0000e+00
Epoch 3/500
2/2 [==============================] - 0s 9ms/step - loss: 2.1112 - accuracy: 0.0328
Epoch 4/500
2/2 [==============================] - 0s 10ms/step - loss: 2.0939 - accuracy: 0.1148
Epoch 5/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0838 - accuracy: 0.1803
Epoch 6/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0724 - accuracy: 0.1475
Epoch 7/500
2/2 [==============================] - 0s 10ms/step - loss: 2.0522 - accuracy: 0.1475
Epoch 8/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0817 - accuracy: 0.1803
Epoch 9/500
2/2 [==============================] - 0s 10ms/step - loss: 2.0922 - accuracy: 0.0984
Epoch 10/500
2/2 [==============================] - 0s 11ms/step - loss: 2.0305 - accuracy: 0.2131
Epoch 11/500
2/2 [===

In [76]:
pickle.dump(model, open("260823_mdl_cbot_df.pkl", "wb"))